In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import zipfile

# Unzip the data folder first

In [2]:
#hc - housing core
#bls - bureal of labor
#ctr - corporate tax rate
#rm -  rent median
#cs - census_starter
#fa - food access
#ue - unemployment
#me - medicare enrollment

df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv') #Done
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx') #Done
df_ctr = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Corporate_tax_rate.csv') #Done
df_rm_FY17 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2017_50_rev.xlsx') #Done
df_rm_FY18 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2018_50_County_rev.xlsx') #Done
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx') #Done
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx') #Done
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx') #Done
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx') #Done
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx') #Done
df_fa = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FoodAccessResearchAtlasData2019.xlsx', sheet_name='Food Access Research Atlas') #Only has 2019 data not usable
df_ue = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\Unemployment.xlsx', skiprows=4) #Cut
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv') #Done


df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv') #Done
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

## Format Census Starter to join into train and test data

In [3]:
df_cs
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


In [4]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
0,1001,2017,55317.0,76.6,14.5,2.1,1.3
1,1001,2018,58786.0,78.9,15.9,2.0,1.1
2,1001,2019,58731.0,80.6,16.1,2.3,0.7
3,1001,2020,57982.0,82.7,16.7,2.3,0.6
4,1001,2021,62660.0,85.5,16.4,2.1,1.1
...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0


### df_hc housing core looks like it is already in a joinable format

## Format bureal of labor to join into train and test data

In [5]:
df_bls2 = pd.pivot_table(df_bls, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,2011-01-01,23440.0,25800.0,2360.0,9.1
1,1001,2011-02-01,23516.0,25850.0,2334.0,9.0
2,1001,2011-03-01,23643.0,25862.0,2219.0,8.6
3,1001,2011-04-01,23787.0,25872.0,2085.0,8.1
4,1001,2011-05-01,23773.0,25863.0,2090.0,8.1
...,...,...,...,...,...,...
87646,17045,2022-06-01,NaN,NaN,318.0,3.6
87647,17045,2022-07-01,NaN,NaN,352.0,3.8
87648,17045,2022-08-01,NaN,NaN,327.0,3.6
87649,17045,2022-09-01,NaN,NaN,281.0,3.2


In [6]:
df_ctr2 = pd.melt(df_ctr, id_vars=['State'])
df_ctr2

,State,variable,value
0,Alabama,2008,4.23%
1,Alaska,2008,9.40%
2,Arizona,2008,6.97%
3,Arkansas,2008,6.50%
4,California,2008,8.84%
...,...,...,...
745,Virginia,2022,7.63%
746,Washington,2022,6.81%
747,West Virginia,2022,6.50%
748,Wisconsin,2022,7.90%


In [7]:
df_me
#No formatting change needed

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [8]:
df_ue2 = pd.melt(df_ue, id_vars=['FIPS_code', 'State', 'Area_name'])
df_ue2['Year'] = df_ue2['variable'].str[-4:]
df_ue2['variable'] = df_ue2['variable'].str[:-5]
df_ue2 = pd.pivot_table(df_ue2, values='value', index=['FIPS_code', 'State', 'Area_name', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_ue2['Year'].max()

'2021'

Unemployment data on a two year lag. Match the 2021 data with 2023 year, 2020 with 2022 and so on

In [9]:
df_me

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
0,2022,April,National,US,National,Total,NaN,64665615,35102417,29563198,...,56614798,7752840,236979,7515861,58785851,29255529,29530322,50013581,23367558,26646023
1,2022,April,State,AL,Alabama,Total,1.0,1078349,495600,582749,...,876402,196610,5000,191610,998869,416218,582651,822177,268695,553482
2,2022,April,County,AL,Alabama,Autauga,1001.0,11492,5150,6342,...,9099,2339,55,2284,10643,4303,6340,7985,1829,6156
3,2022,April,County,AL,Alabama,Baldwin,1003.0,58902,27624,31278,...,51904,6798,131,6667,55097,23830,31267,44841,15131,29710
4,2022,April,County,AL,Alabama,Barbour,1005.0,6260,2440,3820,...,4877,1329,41,1288,5932,2113,3819,5050,1396,3654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,2021,September,County,VI,Virgin Islands,St. John,78020.0,921,745,176,...,*,47,*,*,838,662,176,470,299,171
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030.0,9625,6669,2956,...,8883,669,50,619,8675,5719,2956,5792,2859,2933
60038,2021,September,County,VI,Virgin Islands,Unknown,78999.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*
60039,2021,September,State,UK,Unknown,Unknown,99.0,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [10]:
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [11]:
df_train['first_day_of_month'] = pd.to_datetime(df_train['first_day_of_month'])

In [12]:
df_bls2['first_day_of_month_lag'] = df_bls2['first_day_of_month'] + pd.DateOffset(months=12)

In [13]:
print(df_bls2.dtypes)
print(df_train.dtypes)

measure_type
cfips                              int64
first_day_of_month        datetime64[ns]
Employment                       float64
Labor force                      float64
Unemployment                     float64
Unemployment rate                float64
first_day_of_month_lag    datetime64[ns]
dtype: object
row_id                           object
cfips                             int64
county                           object
state                            object
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
dtype: object


In [14]:
df_train_bls = df_train.merge(df_bls2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag'], indicator=True)

In [15]:
df_bls2['first_day_of_month'].max()

Timestamp('2022-11-01 00:00:00')

In [16]:
df_train_bls

,row_id,cfips,county,state,first_day_of_month_x,microbusiness_density,active,first_day_of_month_y,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2018-08-01,25484.0,26496.0,1012.0,3.8,2019-08-01,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2018-09-01,25443.0,26354.0,911.0,3.5,2019-09-01,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2018-10-01,25614.0,26526.0,912.0,3.4,2019-10-01,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2018-11-01,25639.0,26485.0,846.0,3.2,2019-11-01,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2018-12-01,25664.0,26543.0,879.0,3.3,2019-12-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only


In [17]:
df_train_bls[df_train_bls['Employment'].isnull()]

,row_id,cfips,county,state,first_day_of_month_x,microbusiness_density,active,first_day_of_month_y,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag,_merge
21294,15005_2019-08-01,15005,Kalawao County,Hawaii,2019-08-01,8.333333,7,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21295,15005_2019-09-01,15005,Kalawao County,Hawaii,2019-09-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21296,15005_2019-10-01,15005,Kalawao County,Hawaii,2019-10-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21297,15005_2019-11-01,15005,Kalawao County,Hawaii,2019-11-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
21298,15005_2019-12-01,15005,Kalawao County,Hawaii,2019-12-01,9.523809,8,NaT,NaN,NaN,NaN,NaN,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,NaT,NaN,NaN,NaN,NaN,NaT,left_only


In [18]:
df_test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [19]:
bls_test = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.csv')

In [20]:
bls_test

,series_id,year,period,value,footnote_codes,cfips,first_day_of_month,measure_type
0,LAUCN010010000000003,1990,M01,6.5,NaN,1001,1990-01-01,Unemployment rate
1,LAUCN010010000000003,1990,M02,6.5,NaN,1001,1990-02-01,Unemployment rate
2,LAUCN010010000000003,1990,M03,5.7,NaN,1001,1990-03-01,Unemployment rate
3,LAUCN010010000000003,1990,M04,6.6,NaN,1001,1990-04-01,Unemployment rate
4,LAUCN010010000000003,1990,M05,6.0,NaN,1001,1990-05-01,Unemployment rate
...,...,...,...,...,...,...,...,...
5483039,LAUCN721530000000006,2022,M06,10162.0,NaN,72153,2022-06-01,Labor force
5483040,LAUCN721530000000006,2022,M07,9987.0,NaN,72153,2022-07-01,Labor force
5483041,LAUCN721530000000006,2022,M08,10276.0,P,72153,2022-08-01,Labor force
5483042,LAUCN721530000000006,2022,M09,NaN,N,72153,2022-09-01,Labor force


In [21]:
bls_test['cfips'].unique()


array([ 1001,  1003,  1005, ..., 72149, 72151, 72153], dtype=int64)

In [22]:
df_bls2 = bls_test[bls_test['period'] != 'M13']
#df_bls2['cfips'] = df_bls2['cfips'].astype('object')

In [23]:
df_bls2_2 = pd.pivot_table(df_bls2, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2_2

measure_type,cfips,first_day_of_month,Employment,Labor force,Unemployment,Unemployment rate
0,1001,1990-01-01,15504.0,16582.0,1078.0,6.5
1,1001,1990-02-01,15515.0,16591.0,1076.0,6.5
2,1001,1990-03-01,15768.0,16719.0,951.0,5.7
3,1001,1990-04-01,15787.0,16898.0,1111.0,6.6
4,1001,1990-05-01,15869.0,16889.0,1020.0,6.0
...,...,...,...,...,...,...
1267790,72153,2022-06-01,9202.0,10162.0,960.0,9.4
1267791,72153,2022-07-01,9090.0,9987.0,897.0,9.0
1267792,72153,2022-08-01,9309.0,10276.0,967.0,9.4
1267793,72153,2022-09-01,0.0,0.0,0.0,0.0


In [24]:
df_bls2_2['first_day_of_month'] = pd.to_datetime(df_bls2_2['first_day_of_month'])

In [25]:
df_bls2_2['first_day_of_month_lag_6'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=6)
df_bls2_2['first_day_of_month_lag_7'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=7)
df_bls2_2['first_day_of_month_lag_8'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=8)
df_bls2_2['first_day_of_month_lag_9'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=9)
df_bls2_2['first_day_of_month_lag_10'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=10)
df_bls2_2['first_day_of_month_lag_11'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=11)
df_bls2_2['first_day_of_month_lag_12'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=12)

In [37]:
df_train_bls6 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_6'], 
                              indicator=True, suffixes=['_orig', '_bls'])
#df_train_bls6 = df_train_bls6[df_train_bls6['Employment'].notnull()]

df_train_bls7 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_7'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls7 = df_train_bls7[df_train_bls7['Employment'].notnull()]

df_train_bls8 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_8'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls8 = df_train_bls8[df_train_bls8['Employment'].notnull()]

df_train_bls9 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_9'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls9 = df_train_bls9[df_train_bls9['Employment'].notnull()]

df_train_bls10 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_10'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls10 = df_train_bls10[df_train_bls10['Employment'].notnull()]

df_train_bls11 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_11'], 
                              indicator=True, suffixes=['_orig', '_bls'])
df_train_bls11 = df_train_bls11[df_train_bls11['Employment'].notnull()]

df_train_bls12 = df_train.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_12'], 
                              indicator=True, suffixes=['_orig', '_bls'])
#df_train_bls12 = df_train_bls12[df_train_bls12['Employment'].notnull()]

In [39]:
df_train_bls12[df_train_bls12['Employment'].notnull()]

,row_id,cfips,county,state,first_day_of_month_orig,microbusiness_density,active,first_day_of_month_bls,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag_6,first_day_of_month_lag_7,first_day_of_month_lag_8,first_day_of_month_lag_9,first_day_of_month_lag_10,first_day_of_month_lag_11,first_day_of_month_lag_12,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2018-08-01,25484.0,26496.0,1012.0,3.8,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2018-09-01,25443.0,26354.0,911.0,3.5,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2018-10-01,25614.0,26526.0,912.0,3.4,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2018-11-01,25639.0,26485.0,846.0,3.2,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2018-12-01,25664.0,26543.0,879.0,3.3,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2021-06-01,3620.0,3752.0,132.0,3.5,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,both
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2021-07-01,3529.0,3641.0,112.0,3.1,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,both
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2021-08-01,3633.0,3738.0,105.0,2.8,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,both
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2021-09-01,3623.0,3715.0,92.0,2.5,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,both


In [28]:
#6 month lag seems like the best correlations
df_train_bls6[['microbusiness_density', 'Employment', 'Labor force', 'Unemployment', 'Unemployment rate']].corr()

,microbusiness_density,Employment,Labor force,Unemployment,Unemployment rate
microbusiness_density,1.000000,0.278992,0.276131,0.196738,-0.018429
Employment,0.278992,1.000000,0.999054,0.825855,0.046060
Labor force,0.276131,0.999054,1.000000,0.849600,0.058305
Unemployment,0.196738,0.825855,0.849600,1.000000,0.197360
Unemployment rate,-0.018429,0.046060,0.058305,0.197360,1.000000


In [29]:
import statsmodels.api as sm
from itertools import combinations

vars_list = ['Employment', 'Labor force', 'Unemployment', 'Unemployment rate']
vars_combinations = list(combinations(vars_list, 1)) + list(combinations(vars_list, 2)) + list(combinations(vars_list, 3)) + list(combinations(vars_list, 4))
vars_combinations = [list(tup) for tup in vars_combinations]
print(vars_combinations)

for lists in vars_combinations:
    x = df_train_bls12[lists]
    y = df_train_bls12[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

[['Employment'], ['Labor force'], ['Unemployment'], ['Unemployment rate'], ['Employment', 'Labor force'], ['Employment', 'Unemployment'], ['Employment', 'Unemployment rate'], ['Labor force', 'Unemployment'], ['Labor force', 'Unemployment rate'], ['Unemployment', 'Unemployment rate'], ['Employment', 'Labor force', 'Unemployment'], ['Employment', 'Labor force', 'Unemployment rate'], ['Employment', 'Unemployment', 'Unemployment rate'], ['Labor force', 'Unemployment', 'Unemployment rate'], ['Employment', 'Labor force', 'Unemployment', 'Unemployment rate']]
                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.078
Model:                               OLS   Adj. R-squared:                  0.078
Method:                    Least Squares   F-statistic:                 1.027e+04
Date:                   Sat, 25 Feb 2023   Prob (F-statistic):               0.00
Time:                          

From these OLS Regression results we have decided to keep Employment, Unemployment, and Unemployment rate

In [391]:
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [392]:
df_cs2['year_lag'] = (pd.to_datetime(df_cs2['Year']) + pd.DateOffset(years=2)).dt.year
df_cs2

variable,cfips,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag
0,1001,2017,55317.0,76.6,14.5,2.1,1.3,2019
1,1001,2018,58786.0,78.9,15.9,2.0,1.1,2020
2,1001,2019,58731.0,80.6,16.1,2.3,0.7,2021
3,1001,2020,57982.0,82.7,16.7,2.3,0.6,2022
4,1001,2021,62660.0,85.5,16.4,2.1,1.1,2023
...,...,...,...,...,...,...,...,...
15705,56045,2017,59605.0,71.1,14.1,3.8,0.6,2019
15706,56045,2018,52867.0,73.3,13.5,4.1,0.6,2020
15707,56045,2019,57031.0,76.8,13.4,1.7,0.0,2021
15708,56045,2020,53333.0,79.7,12.7,2.3,0.0,2022


In [393]:
df_train['year'] = df_train['first_day_of_month'].dt.year
df_train_cs = df_train.merge(df_cs2, how='left', left_on=['cfips', 'year'], right_on=['cfips','year_lag'], 
                              indicator=True, suffixes=['_orig', '_cps'])
df_train_cs

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,2017,55317.0,76.6,14.5,2.1,1.3,2019,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,2017,55317.0,76.6,14.5,2.1,1.3,2019,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2017,55317.0,76.6,14.5,2.1,1.3,2019,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,2017,55317.0,76.6,14.5,2.1,1.3,2019,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,2017,55317.0,76.6,14.5,2.1,1.3,2019,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,2020,53333.0,79.7,12.7,2.3,0.0,2022,both
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,2020,53333.0,79.7,12.7,2.3,0.0,2022,both
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,2020,53333.0,79.7,12.7,2.3,0.0,2022,both
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,2020,53333.0,79.7,12.7,2.3,0.0,2022,both


In [394]:
df_train_cs[df_train_cs['median_hh_inc'].isnull()]

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag,_merge


In [395]:
df_train_cs[['microbusiness_density', 'median_hh_inc','pct_bb','pct_college','pct_foreign_born','pct_it_workers']].corr()

,microbusiness_density,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers
microbusiness_density,1.000000,0.378204,0.359595,0.480311,0.277879,0.254915
median_hh_inc,0.378204,1.000000,0.732426,0.708595,0.375235,0.312024
pct_bb,0.359595,0.732426,1.000000,0.656127,0.281933,0.282487
pct_college,0.480311,0.708595,0.656127,1.000000,0.327771,0.371154
pct_foreign_born,0.277879,0.375235,0.281933,0.327771,1.000000,0.210967
pct_it_workers,0.254915,0.312024,0.282487,0.371154,0.210967,1.000000


In [396]:
vars_list = ['median_hh_inc','pct_bb','pct_college','pct_foreign_born','pct_it_workers']
vars_combinations = list(combinations(vars_list, 4)) \
                    + list(combinations(vars_list, 5))
vars_combinations = [list(tup) for tup in vars_combinations]
print(vars_combinations)

for lists in vars_combinations:
    x = df_train_cs[lists]
    y = df_train_cs[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

[['median_hh_inc'], ['pct_bb'], ['pct_college'], ['pct_foreign_born'], ['pct_it_workers'], ['median_hh_inc', 'pct_bb'], ['median_hh_inc', 'pct_college'], ['median_hh_inc', 'pct_foreign_born'], ['median_hh_inc', 'pct_it_workers'], ['pct_bb', 'pct_college'], ['pct_bb', 'pct_foreign_born'], ['pct_bb', 'pct_it_workers'], ['pct_college', 'pct_foreign_born'], ['pct_college', 'pct_it_workers'], ['pct_foreign_born', 'pct_it_workers'], ['median_hh_inc', 'pct_bb', 'pct_college'], ['median_hh_inc', 'pct_bb', 'pct_foreign_born'], ['median_hh_inc', 'pct_bb', 'pct_it_workers'], ['median_hh_inc', 'pct_college', 'pct_foreign_born'], ['median_hh_inc', 'pct_college', 'pct_it_workers'], ['median_hh_inc', 'pct_foreign_born', 'pct_it_workers'], ['pct_bb', 'pct_college', 'pct_foreign_born'], ['pct_bb', 'pct_college', 'pct_it_workers'], ['pct_bb', 'pct_foreign_born', 'pct_it_workers'], ['pct_college', 'pct_foreign_born', 'pct_it_workers'], ['median_hh_inc', 'pct_bb', 'pct_college', 'pct_foreign_born'], ['med

#### we are going to drop median_hh_inc from the analysis due to the .5 p value with the other variables and no increase to R^2 with it inclusion. While median_hh_inc by itself gave a p-value of >.001 its information may be given by the other values which is why the p-value increased with everything together. Also the R^2 did not improve with the inclusion of median_hh_inc.

In [397]:
print(df_ctr2.dtypes)
d['value'] = df_ctr2['value'].str.rstrip('%').astype('float') / 100.0
df_ctr2['year_lag_1'] = (pd.to_datetime(df_ctr2['variable']) + pd.DateOffset(years=1)).dt.year
df_ctr2['year_lag_2'] = (pd.to_datetime(df_ctr2['variable']) + pd.DateOffset(years=2)).dt.year
df_ctr2

State       object
variable    object
value       object
dtype: object


,State,variable,value,year_lag_1,year_lag_2
0,Alabama,2008,0.0423,2009,2010
1,Alaska,2008,0.0940,2009,2010
2,Arizona,2008,0.0697,2009,2010
3,Arkansas,2008,0.0650,2009,2010
4,California,2008,0.0884,2009,2010
...,...,...,...,...,...
745,Virginia,2022,0.0763,2023,2024
746,Washington,2022,0.0681,2023,2024
747,West Virginia,2022,0.0650,2023,2024
748,Wisconsin,2022,0.0790,2023,2024


In [398]:
df_train['state'] = df_train['state'].astype(str)
df_ctr2['State'] = df_ctr2['State'].str[:-1]
print(df_train['state'].unique())
print(df_ctr2['State'].unique())
#print(df_ctr2['State'].str[:-1])

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming']
['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio

In [399]:
df_train_ctr_1 = df_train.merge(df_ctr2, how='left', left_on=['state', 'year'], right_on=['State','year_lag_1'], 
                              indicator=True, suffixes=['_orig', '_ctr'])
df_train_ctr_1
df_train_ctr_2 = df_train.merge(df_ctr2, how='left', left_on=['state', 'year'], right_on=['State','year_lag_2'], 
                              indicator=True, suffixes=['_orig', '_ctr'])
df_train_ctr_2


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,State,variable,value,year_lag_1,year_lag_2,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,Alabama,2017,0.0423,2018.0,2019.0,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,Alabama,2017,0.0423,2018.0,2019.0,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,Alabama,2017,0.0423,2018.0,2019.0,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,Alabama,2017,0.0423,2018.0,2019.0,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,Alabama,2017,0.0423,2018.0,2019.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,Wyoming,2020,0.0000,2021.0,2022.0,both
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,Wyoming,2020,0.0000,2021.0,2022.0,both
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,Wyoming,2020,0.0000,2021.0,2022.0,both
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,Wyoming,2020,0.0000,2021.0,2022.0,both


In [400]:
df_train_ctr_1 = df_train_ctr_1[df_train_ctr_1['value'].notnull()]
df_train_ctr_2 = df_train_ctr_2[df_train_ctr_2['value'].notnull()]

In [401]:
print(df_train_ctr_1[['microbusiness_density', 'value']].corr())
print(df_train_ctr_2[['microbusiness_density', 'value']].corr())

                       microbusiness_density     value
microbusiness_density               1.000000  0.018312
value                               0.018312  1.000000
                       microbusiness_density     value
microbusiness_density               1.000000  0.014425
value                               0.014425  1.000000


In [402]:
vars_list = ['value']
vars_combinations = list(combinations(vars_list, 1))
vars_combinations = [list(tup) for tup in vars_combinations]
print(vars_combinations)

for lists in vars_combinations:
    x = df_train_ctr_1[lists]
    y = df_train_ctr_1[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

for lists in vars_combinations:
    x = df_train_ctr_2[lists]
    y = df_train_ctr_2[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

[['value']]
                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.000
Model:                               OLS   Adj. R-squared:                  0.000
Method:                    Least Squares   F-statistic:                     41.00
Date:                   Sun, 12 Feb 2023   Prob (F-statistic):           1.53e-10
Time:                           19:45:48   Log-Likelihood:            -3.6985e+05
No. Observations:                 122226   AIC:                         7.397e+05
Df Residuals:                     122224   BIC:                         7.397e+05
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const     

Corporate tax rate has an R^2 of 0 so we should not use it even though there was a indication of correlation

In [403]:
df_hc

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,202211,39123,"ottawa, oh",357150.0,0.0225,0.1092,112.0,-0.0345,-0.0088,62.0,...,-0.0057,473973.0,0.0360,-0.0019,178.0,0.0260,-0.1706,0.6027,0.0941,-0.3354
1,202211,13291,"union, ga",451000.0,-0.0196,0.1000,156.0,-0.0189,0.3333,71.0,...,-0.0188,537786.0,-0.0170,-0.1077,220.0,-0.0738,0.0733,0.4071,-0.0961,-0.3750
2,202211,18095,"madison, in",187700.0,0.0434,0.1925,256.0,-0.0376,0.3690,46.0,...,0.0608,211245.0,0.0300,0.1171,437.0,-0.0702,-0.0343,0.7129,-0.0803,-0.7176
3,202211,13193,"macon, ga",161250.0,-0.2833,-0.0706,13.0,0.1818,-0.1613,72.0,...,-0.1472,214887.0,-0.2069,-0.1152,13.0,0.1818,-0.3158,0.0000,-0.0909,-0.3226
4,202211,21061,"edmonson, ky",357500.0,-0.1005,0.5383,22.0,-0.1042,-0.0444,61.0,...,0.0759,431635.0,-0.0451,0.0650,36.0,-0.0405,-0.2680,0.6512,0.1095,-0.5488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238800,201607,55097,"portage, wi",169900.0,NaN,NaN,323.0,NaN,NaN,65.0,...,NaN,215669.0,NaN,NaN,326.0,NaN,NaN,0.0124,NaN,NaN
238801,201607,25027,"worcester, ma",299900.0,NaN,NaN,2703.0,NaN,NaN,60.0,...,NaN,365063.0,NaN,NaN,3692.0,NaN,NaN,0.3727,NaN,NaN
238802,201607,38035,"grand forks, nd",247400.0,NaN,NaN,273.0,NaN,NaN,60.0,...,NaN,279366.0,NaN,NaN,274.0,NaN,NaN,0.0037,NaN,NaN
238803,201607,48413,"schleicher, tx",73100.0,NaN,NaN,2.0,NaN,NaN,430.0,...,NaN,73100.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [404]:
df_train['year_month'] = df_train['first_day_of_month'].dt.year.apply(str) + df_train['first_day_of_month'].dt.month.apply(str).str.zfill(2)
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,year_month
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,201909
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,201910
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,201911
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,201912
...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,202206
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,202207
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,202209


In [405]:
df_train_ctr_1 = df_train.merge(df_ctr2, how='left', left_on=['state', 'year'], right_on=['State','year_lag_1'], 
                              indicator=True, suffixes=['_orig', '_ctr'])

In [406]:
df_hc['month_date_yyyymm'].apply(str).str[0:4]

0         2022
1         2022
2         2022
3         2022
4         2022
          ... 
238800    2016
238801    2016
238802    2016
238803    2016
238804    2016
Name: month_date_yyyymm, Length: 238805, dtype: object

In [407]:
df_hc['date'] = pd.to_datetime(df_hc['month_date_yyyymm'].apply(str).str[0:4]+'-'+df_hc['month_date_yyyymm'].apply(str).str[4:6]+'-01')
df_hc

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,date
0,202211,39123,"ottawa, oh",357150.0,0.0225,0.1092,112.0,-0.0345,-0.0088,62.0,...,473973.0,0.0360,-0.0019,178.0,0.0260,-0.1706,0.6027,0.0941,-0.3354,2022-11-01
1,202211,13291,"union, ga",451000.0,-0.0196,0.1000,156.0,-0.0189,0.3333,71.0,...,537786.0,-0.0170,-0.1077,220.0,-0.0738,0.0733,0.4071,-0.0961,-0.3750,2022-11-01
2,202211,18095,"madison, in",187700.0,0.0434,0.1925,256.0,-0.0376,0.3690,46.0,...,211245.0,0.0300,0.1171,437.0,-0.0702,-0.0343,0.7129,-0.0803,-0.7176,2022-11-01
3,202211,13193,"macon, ga",161250.0,-0.2833,-0.0706,13.0,0.1818,-0.1613,72.0,...,214887.0,-0.2069,-0.1152,13.0,0.1818,-0.3158,0.0000,-0.0909,-0.3226,2022-11-01
4,202211,21061,"edmonson, ky",357500.0,-0.1005,0.5383,22.0,-0.1042,-0.0444,61.0,...,431635.0,-0.0451,0.0650,36.0,-0.0405,-0.2680,0.6512,0.1095,-0.5488,2022-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238800,201607,55097,"portage, wi",169900.0,NaN,NaN,323.0,NaN,NaN,65.0,...,215669.0,NaN,NaN,326.0,NaN,NaN,0.0124,NaN,NaN,2016-07-01
238801,201607,25027,"worcester, ma",299900.0,NaN,NaN,2703.0,NaN,NaN,60.0,...,365063.0,NaN,NaN,3692.0,NaN,NaN,0.3727,NaN,NaN,2016-07-01
238802,201607,38035,"grand forks, nd",247400.0,NaN,NaN,273.0,NaN,NaN,60.0,...,279366.0,NaN,NaN,274.0,NaN,NaN,0.0037,NaN,NaN,2016-07-01
238803,201607,48413,"schleicher, tx",73100.0,NaN,NaN,2.0,NaN,NaN,430.0,...,73100.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2016-07-01


In [408]:
df_hc['date_lag_7'] = df_hc['date'] + pd.DateOffset(months=7)
df_hc['date_lag_8'] = df_hc['date'] + pd.DateOffset(months=8)
df_hc['date_lag_9'] = df_hc['date'] + pd.DateOffset(months=9)
df_hc['date_lag_10'] = df_hc['date'] + pd.DateOffset(months=10)
df_hc['date_lag_11'] = df_hc['date'] + pd.DateOffset(months=11)
df_hc['date_lag_12'] = df_hc['date'] + pd.DateOffset(months=12)

In [409]:
df_train_hc7 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_7'], 
                              indicator=True, suffixes=['_orig', '_hc'])

df_train_hc8 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_8'], 
                              indicator=True, suffixes=['_orig', '_hc'])

df_train_hc9 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_9'], 
                              indicator=True, suffixes=['_orig', '_hc'])

df_train_hc10 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_10'], 
                              indicator=True, suffixes=['_orig', '_hc'])

df_train_hc11 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_11'], 
                              indicator=True, suffixes=['_orig', '_hc'])

df_train_hc12 = df_train.merge(df_hc, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['county_fips','date_lag_12'], 
                              indicator=True, suffixes=['_orig', '_hc'])

In [410]:
df_train_hc6

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year_month,month_date_yyyymm,county_fips,...,pending_ratio_yy,date,date_lag_6,date_lag_7,date_lag_8,date_lag_9,date_lag_10,date_lag_11,date_lag_12,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,201908,201902.0,1001.0,...,0.0972,2019-02-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,201909,201903.0,1001.0,...,0.1677,2019-03-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,201910,201904.0,1001.0,...,0.2992,2019-04-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,201911,201905.0,1001.0,...,0.2817,2019-05-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,201912,201906.0,1001.0,...,0.1170,2019-06-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,202206,202112.0,56045.0,...,-0.0569,2021-12-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,both
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,202207,202201.0,56045.0,...,0.0264,2022-01-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,both
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,202208,202202.0,56045.0,...,-0.0234,2022-02-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,both
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,202209,202203.0,56045.0,...,-0.0560,2022-03-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,both


In [411]:
hc_values = ['median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']

In [412]:
df_train_hc7_a = df_train_hc7[df_train_hc7.isnull().any(axis=1)]
df_train_hc7_test[['first_day_of_month', 'cfips', 'date_lag_7', 'microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 'new_listing_count_mm', 'new_listing_count_yy',
       'price_reduced_count', 'price_reduced_count_mm',
       'price_reduced_count_yy', 'pending_listing_count',
       'pending_listing_count_mm', 'pending_listing_count_yy']]

,first_day_of_month,cfips,date_lag_7,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,...,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy
39,2019-08-01,1003,2019-08-01,7.239156,325253.0,-0.0114,0.0325,2596.0,-0.0204,0.0068,...,-0.0142,822.0,0.5110,0.2305,540.0,0.2385,0.3300,1.0,0.0000,NaN
40,2019-09-01,1003,2019-09-01,7.290936,327000.0,0.0054,0.0383,2674.0,0.0300,0.0092,...,-0.0682,866.0,0.0535,0.0384,486.0,-0.1000,0.1408,1.0,0.0000,NaN
41,2019-10-01,1003,2019-10-01,7.425439,331357.0,0.0133,0.0643,2748.0,0.0277,0.0170,...,-0.1169,860.0,-0.0069,-0.1189,464.0,-0.0453,-0.0720,1.0,0.0000,NaN
43,2019-12-01,1003,2019-12-01,7.470274,339900.0,0.0154,0.0116,2776.0,-0.0139,0.0095,...,-0.0570,850.0,-0.0618,-0.0341,676.0,0.2472,0.1901,NaN,NaN,NaN
44,2020-01-01,1003,2020-01-01,7.413655,344000.0,0.0121,0.0148,2749.0,-0.0095,0.0040,...,-0.0247,764.0,-0.1012,-0.1549,636.0,-0.0592,0.0461,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122255,2022-01-01,56045,2022-01-01,1.749688,201450.0,0.0078,0.1682,45.0,0.0854,-0.2458,...,-0.3125,12.0,0.5000,-0.5000,0.0,NaN,-1.0000,17.0,0.0000,0.4783
122256,2022-02-01,56045,2022-02-01,1.749688,199950.0,-0.0074,0.1769,50.0,0.1236,-0.2063,...,-0.0492,24.0,1.0000,0.2000,12.0,NaN,0.2000,13.0,-0.2353,0.2381
122261,2022-07-01,56045,2022-07-01,1.803249,179900.0,0.0000,0.1164,33.0,-0.1200,-0.3053,...,0.2045,0.0,-1.0000,-1.0000,2.0,-0.5000,NaN,8.0,-0.3478,-0.4444
122262,2022-08-01,56045,2022-08-01,1.785395,205000.0,0.1395,0.2432,28.0,-0.1515,-0.3913,...,0.1045,0.0,NaN,-1.0000,0.0,-1.0000,-1.0000,5.0,-0.3333,-0.2857


In [413]:
for i in hc_values:
    print(i, ': ', df_train_hc7[i].isna().sum())

median_listing_price :  1380
median_listing_price_mm :  1593
median_listing_price_yy :  1981
active_listing_count :  1373
active_listing_count_mm :  1629
active_listing_count_yy :  2023
median_days_on_market :  1381
new_listing_count :  1371
price_reduced_count :  1371


In [414]:
df_train_hc_7r = df_train_hc7[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

df_train_hc_8r = df_train_hc8[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

df_train_hc_9r = df_train_hc9[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

df_train_hc_10r = df_train_hc10[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

df_train_hc_11r = df_train_hc11[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

df_train_hc_12r = df_train_hc12[['microbusiness_density', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count',
       'price_reduced_count']].dropna()

In [415]:
df_train_hc_7r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.022136,0.000425,-0.000083,0.283714,-0.015964,-0.031281,-0.113184,0.307520,0.256829
median_listing_price,0.022136,1.000000,0.705221,0.997669,0.008398,-0.006051,-0.005319,-0.008995,0.010207,0.006754
median_listing_price_mm,0.000425,0.705221,1.000000,0.704762,-0.001104,-0.000612,-0.002169,-0.002839,-0.001114,-0.000959
median_listing_price_yy,-0.000083,0.997669,0.704762,1.000000,-0.003304,-0.005134,-0.002525,-0.001528,-0.003095,-0.002940
active_listing_count,0.283714,0.008398,-0.001104,-0.003304,1.000000,-0.002471,0.022655,-0.094115,0.872731,0.923262
active_listing_count_mm,-0.015964,-0.006051,-0.000612,-0.005134,-0.002471,1.000000,0.161285,-0.164367,0.010289,0.008226
active_listing_count_yy,-0.031281,-0.005319,-0.002169,-0.002525,0.022655,0.161285,1.000000,0.074961,-0.012720,0.029426
median_days_on_market,-0.113184,-0.008995,-0.002839,-0.001528,-0.094115,-0.164367,0.074961,1.000000,-0.170189,-0.116233
new_listing_count,0.307520,0.010207,-0.001114,-0.003095,0.872731,0.010289,-0.012720,-0.170189,1.000000,0.866684
price_reduced_count,0.256829,0.006754,-0.000959,-0.002940,0.923262,0.008226,0.029426,-0.116233,0.866684,1.000000


In [416]:
df_train_hc_8r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.021939,0.000505,-0.000053,0.285434,-0.020932,-0.030434,-0.113261,0.306729,0.258384
median_listing_price,0.021939,1.000000,0.705247,0.997743,0.008441,-0.006340,-0.005418,-0.009054,0.010160,0.006827
median_listing_price_mm,0.000505,0.705247,1.000000,0.704804,-0.001105,-0.000586,-0.002173,-0.002894,-0.001103,-0.000953
median_listing_price_yy,-0.000053,0.997743,0.704804,1.000000,-0.003258,-0.005103,-0.002607,-0.001743,-0.003033,-0.002882
active_listing_count,0.285434,0.008441,-0.001105,-0.003258,1.000000,-0.003629,0.025900,-0.096274,0.872998,0.924389
active_listing_count_mm,-0.020932,-0.006340,-0.000586,-0.005103,-0.003629,1.000000,0.162026,-0.159610,0.008950,0.007509
active_listing_count_yy,-0.030434,-0.005418,-0.002173,-0.002607,0.025900,0.162026,1.000000,0.081607,-0.012008,0.031768
median_days_on_market,-0.113261,-0.009054,-0.002894,-0.001743,-0.096274,-0.159610,0.081607,1.000000,-0.171252,-0.118679
new_listing_count,0.306729,0.010160,-0.001103,-0.003033,0.872998,0.008950,-0.012008,-0.171252,1.000000,0.868272
price_reduced_count,0.258384,0.006827,-0.000953,-0.002882,0.924389,0.007509,0.031768,-0.118679,0.868272,1.000000


In [417]:
df_train_hc_9r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.021528,0.000503,-0.000229,0.287526,-0.021546,-0.029084,-0.113660,0.306779,0.259815
median_listing_price,0.021528,1.000000,0.705269,0.997813,0.008509,-0.006481,-0.005440,-0.009242,0.010146,0.006896
median_listing_price_mm,0.000503,0.705269,1.000000,0.704854,-0.001107,-0.000598,-0.002157,-0.002975,-0.001097,-0.000947
median_listing_price_yy,-0.000229,0.997813,0.704854,1.000000,-0.003213,-0.005064,-0.002485,-0.002185,-0.002976,-0.002822
active_listing_count,0.287526,0.008509,-0.001107,-0.003213,1.000000,-0.004848,0.027880,-0.098119,0.874921,0.924260
active_listing_count_mm,-0.021546,-0.006481,-0.000598,-0.005064,-0.004848,1.000000,0.157716,-0.161607,0.009157,0.006418
active_listing_count_yy,-0.029084,-0.005440,-0.002157,-0.002485,0.027880,0.157716,1.000000,0.083290,-0.010477,0.033966
median_days_on_market,-0.113660,-0.009242,-0.002975,-0.002185,-0.098119,-0.161607,0.083290,1.000000,-0.171642,-0.119976
new_listing_count,0.306779,0.010146,-0.001097,-0.002976,0.874921,0.009157,-0.010477,-0.171642,1.000000,0.868254
price_reduced_count,0.259815,0.006896,-0.000947,-0.002822,0.924260,0.006418,0.033966,-0.119976,0.868254,1.000000


In [418]:
df_train_hc_10r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.021139,0.000237,-0.000410,0.289734,-0.018528,-0.028601,-0.114711,0.307403,0.261099
median_listing_price,0.021139,1.000000,0.705288,0.997845,0.008585,-0.006320,-0.005480,-0.009332,0.010167,0.006958
median_listing_price_mm,0.000237,0.705288,1.000000,0.704868,-0.001119,-0.000671,-0.002163,-0.002979,-0.001101,-0.000952
median_listing_price_yy,-0.000410,0.997845,0.704868,1.000000,-0.003182,-0.005106,-0.002497,-0.002280,-0.002935,-0.002776
active_listing_count,0.289734,0.008585,-0.001119,-0.003182,1.000000,-0.006072,0.028876,-0.099703,0.877147,0.922663
active_listing_count_mm,-0.018528,-0.006320,-0.000671,-0.005106,-0.006072,1.000000,0.157403,-0.166313,0.009882,0.005566
active_listing_count_yy,-0.028601,-0.005480,-0.002163,-0.002497,0.028876,0.157403,1.000000,0.083069,-0.009048,0.035501
median_days_on_market,-0.114711,-0.009332,-0.002979,-0.002280,-0.099703,-0.166313,0.083069,1.000000,-0.172374,-0.120693
new_listing_count,0.307403,0.010167,-0.001101,-0.002935,0.877147,0.009882,-0.009048,-0.172374,1.000000,0.867012
price_reduced_count,0.261099,0.006958,-0.000952,-0.002776,0.922663,0.005566,0.035501,-0.120693,0.867012,1.000000


In [419]:
df_train_hc_11r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.020944,0.000234,-0.000386,0.291703,-0.013836,-0.028358,-0.116201,0.308474,0.262288
median_listing_price,0.020944,1.000000,0.705307,0.997887,0.008658,-0.006066,-0.005557,-0.009430,0.010235,0.007018
median_listing_price_mm,0.000234,0.705307,1.000000,0.704889,-0.001131,-0.000729,-0.002175,-0.002994,-0.001105,-0.000959
median_listing_price_yy,-0.000386,0.997887,0.704889,1.000000,-0.003142,-0.005138,-0.002725,-0.002346,-0.002883,-0.002722
active_listing_count,0.291703,0.008658,-0.001131,-0.003142,1.000000,-0.005843,0.029777,-0.102049,0.878602,0.921152
active_listing_count_mm,-0.013836,-0.006066,-0.000729,-0.005138,-0.005843,1.000000,0.159309,-0.171976,0.011096,0.006158
active_listing_count_yy,-0.028358,-0.005557,-0.002175,-0.002725,0.029777,0.159309,1.000000,0.083606,-0.007825,0.036574
median_days_on_market,-0.116201,-0.009430,-0.002994,-0.002346,-0.102049,-0.171976,0.083606,1.000000,-0.174039,-0.122498
new_listing_count,0.308474,0.010235,-0.001105,-0.002883,0.878602,0.011096,-0.007825,-0.174039,1.000000,0.866052
price_reduced_count,0.262288,0.007018,-0.000959,-0.002722,0.921152,0.006158,0.036574,-0.122498,0.866052,1.000000


In [420]:
df_train_hc_12r.corr()

,microbusiness_density,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count
microbusiness_density,1.000000,0.020721,0.000226,-0.000432,0.293292,-0.011643,-0.028812,-0.117180,0.308874,0.262902
median_listing_price,0.020721,1.000000,0.705323,0.997918,0.008716,-0.005892,-0.005640,-0.009483,0.010250,0.007046
median_listing_price_mm,0.000226,0.705323,1.000000,0.704901,-0.001141,-0.000764,-0.002174,-0.003022,-0.001108,-0.000963
median_listing_price_yy,-0.000432,0.997918,0.704901,1.000000,-0.003098,-0.005189,-0.002857,-0.002399,-0.002829,-0.002666
active_listing_count,0.293292,0.008716,-0.001141,-0.003098,1.000000,-0.005585,0.029798,-0.104953,0.880299,0.919676
active_listing_count_mm,-0.011643,-0.005892,-0.000764,-0.005189,-0.005585,1.000000,0.161395,-0.178748,0.012496,0.007345
active_listing_count_yy,-0.028812,-0.005640,-0.002174,-0.002857,0.029798,0.161395,1.000000,0.083553,-0.007200,0.036512
median_days_on_market,-0.117180,-0.009483,-0.003022,-0.002399,-0.104953,-0.178748,0.083553,1.000000,-0.175866,-0.124864
new_listing_count,0.308874,0.010250,-0.001108,-0.002829,0.880299,0.012496,-0.007200,-0.175866,1.000000,0.866388
price_reduced_count,0.262902,0.007046,-0.000963,-0.002666,0.919676,0.007345,0.036512,-0.124864,0.866388,1.000000


In [421]:
vars_list = ['active_listing_count',
       'active_listing_count_yy', 'median_days_on_market',
       'new_listing_count', 'price_reduced_count']
print(hc_values)
vars_combinations = list(combinations(vars_list, 5))
vars_combinations = [list(tup) for tup in vars_combinations]
#print(vars_combinations)

print('df_train_hc_7r')
for lists in vars_combinations:
    x = df_train_hc_7r[lists]
    y = df_train_hc_7r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

print('df_train_hc_8r')
for lists in vars_combinations:
    x = df_train_hc_8r[lists]
    y = df_train_hc_8r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

print('df_train_hc_9r')
for lists in vars_combinations:
    x = df_train_hc_9r[lists]
    y = df_train_hc_9r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

print('df_train_hc_10r')
for lists in vars_combinations:
    x = df_train_hc_10r[lists]
    y = df_train_hc_10r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

print('df_train_hc_11r')
for lists in vars_combinations:
    x = df_train_hc_11r[lists]
    y = df_train_hc_11r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

print('df_train_hc_12r')    
for lists in vars_combinations:
    x = df_train_hc_12r[lists]
    y = df_train_hc_12r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

['median_listing_price', 'median_listing_price_mm', 'median_listing_price_yy', 'active_listing_count', 'active_listing_count_mm', 'active_listing_count_yy', 'median_days_on_market', 'new_listing_count', 'price_reduced_count']
df_train_hc_7r
                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.104
Model:                               OLS   Adj. R-squared:                  0.104
Method:                    Least Squares   F-statistic:                     2778.
Date:                   Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                           19:45:50   Log-Likelihood:            -3.5750e+05
No. Observations:                 120053   AIC:                         7.150e+05
Df Residuals:                     120047   BIC:                         7.151e+05
Df Model:                              5                                         
Covariance Type:     

After going through all the variables and looking at multiple regression analysis we found that variables worth keeping from df_hc are 'active_listing_count',
       'active_listing_count_yy', 'median_days_on_market', 'new_listing_count', 'price_reduced_count' with a year long lag

In [422]:
df_rm_FY17['fips'] = df_rm_FY17['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY17['year'] = 2017
df_rm_FY17 = df_rm_FY17.rename(columns={'Rent50_0': 'rent50_0', 'Rent50_1': 'rent50_1', 'Rent50_2': 'rent50_2', 'Rent50_3': 'rent50_3', 'Rent50_4': 'rent50_4'})
df_rm_FY17_keep = df_rm_FY17[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY18['fips'] = df_rm_FY18['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY18['year'] = 2018
df_rm_FY18_keep = df_rm_FY18[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY19['fips'] = df_rm_FY19['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY19['year'] = 2019
df_rm_FY19_keep = df_rm_FY19[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY20['fips'] = df_rm_FY20['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY20['year'] = 2020
df_rm_FY20_keep = df_rm_FY20[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY21['fips'] = df_rm_FY21['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY21['year'] = 2021
df_rm_FY21_keep = df_rm_FY21[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY22['fips'] = df_rm_FY22['state_code'].astype(str) + df_rm_FY22['county_code'].astype(str).str.zfill(3)
df_rm_FY22['fips'] = df_rm_FY22['fips'].astype(int)
df_rm_FY22['year'] = 2022
df_rm_FY22 = df_rm_FY22.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY22_keep = df_rm_FY22[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY23['fips'] = df_rm_FY23['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY23['year'] = 2023
df_rm_FY23 = df_rm_FY23.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY23_keep = df_rm_FY23[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm = pd.concat([df_rm_FY17_keep, df_rm_FY18_keep, df_rm_FY19_keep, df_rm_FY20_keep, df_rm_FY21_keep, df_rm_FY22_keep, df_rm_FY23_keep])
df_rm['year_lag_1'] = df_rm['year'] + 1
df_rm['year_lag_2'] = df_rm['year'] + 2
df_rm

,fips,year,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,year_lag_1,year_lag_2
0,1001,2017,633,735,886,1136,1536,2018,2019
1,1003,2017,865,908,1047,1524,1751,2018,2019
2,1005,2017,540,543,722,903,1031,2018,2019
3,1007,2017,719,812,937,1258,1405,2018,2019
4,1009,2017,719,812,937,1258,1405,2018,2019
...,...,...,...,...,...,...,...,...,...
4759,72133,2023,411,421,475,659,728,2024,2025
4760,72147,2023,411,421,475,659,728,2024,2025
4761,78010,2023,1038,1059,1293,1574,1737,2024,2025
4762,78020,2023,1378,1633,2029,2469,2726,2024,2025


In [423]:
df_rm_FY17

,fips2010,fips2000,rent50_2,rent50_0,rent50_1,rent50_3,rent50_4,State,cbsasub,areaname,county,CouSub,countyname,county_town_name,pop2010,hu2010,state_alpha,fips,year
0,100199999,1.002000e+08,886,633,735,1136,1536,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,22135.0,AL,1001,2017
1,100399999,1.004000e+08,1047,865,908,1524,1751,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,104061.0,AL,1003,2017
2,100599999,1.006000e+08,722,540,543,903,1031,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,11829.0,AL,1005,2017
3,100799999,1.008000e+08,937,719,812,1258,1405,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,8981.0,AL,1007,2017
4,100999999,1.010000e+08,937,719,812,1258,1405,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,23887.0,AL,1009,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764,7215199999,7.215200e+09,605,471,502,813,1011,72,METRO41980MM7440,"San Juan-Guaynabo, PR HUD Metro FMR Area",151.0,99999,Yabucoa Municipio,Yabucoa Municipio,37941.0,15798.0,PR,72151,2017
4765,7215399999,7.215400e+09,432,357,374,555,761,72,METRO38660M49500,"Yauco, PR HUD Metro FMR Area",153.0,99999,Yauco Municipio,Yauco Municipio,42043.0,18020.0,PR,72153,2017
4766,7801099999,7.801100e+09,952,764,780,1181,1305,78,NCNTY78010N78010,"St. Croix Island, VI",10.0,99999,St. Croix,St. Croix,53234.0,25275.0,VI,78010,2017
4767,7802099999,7.802100e+09,1529,1038,1230,1897,2096,78,NCNTY78020N78020,"St. John Island, VI",20.0,99999,St. John,St. John,4197.0,3453.0,VI,78020,2017


In [424]:
df_rm_FY22['fips'] = df_rm_FY22['state_code'].astype(str) + df_rm_FY22['county_code'].astype(str).str.zfill(3)
df_rm_FY22

,state_code,county_code,county_sub_code,cntyname,town_name,areaname22,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,cbsasub22,pop2017,fips,year
0,48,59,99999,Callahan County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,13660,48059,2022
1,48,253,99999,Jones County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,19970,48253,2022
2,48,441,99999,Taylor County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,135370,48441,2022
3,72,3,99999,Aguada Municipio,NaN,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,39470,72003,2022
4,72,5,99999,Aguadilla Municipio,NaN,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,55720,72005,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4760,72,133,99999,Santa Isabel Municipio,NaN,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,22475,72133,2022
4761,72,147,99999,Vieques Municipio,NaN,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,8930,72147,2022
4762,78,10,99999,St. Croix,NaN,"St. Croix Island, VI",964,984,1201,1485,1629,NCNTY78010N78010,53234,78010,2022
4763,78,20,99999,St. John,NaN,"St. John Island, VI",1280,1517,1885,2330,2556,NCNTY78020N78020,4197,78020,2022


In [425]:
df_rm.dtypes

fips          int32
year          int64
rent50_0      int64
rent50_1      int64
rent50_2      int64
rent50_3      int64
rent50_4      int64
year_lag_1    int64
year_lag_2    int64
dtype: object

In [426]:
df_train['year'] = df_train['year_month'].astype(str).str[0:4].astype(int)
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,year_month
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,201909
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,201910
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,201911
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,201912
...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,202206
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,202207
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,202209


In [427]:
df_train.dtypes

row_id                           object
cfips                             int64
county                           object
state                            object
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
year                              int32
year_month                       object
dtype: object

In [428]:
df_train_rm = df_train.merge(df_rm, how='left', left_on=['cfips', 'year'], right_on=['fips','year'], 
                              indicator=True, suffixes=['_orig', 'rm'])
df_train_rm_1 = df_train.merge(df_rm, how='left', left_on=['cfips', 'year'], right_on=['fips','year_lag_1'], 
                              indicator=True, suffixes=['_orig', 'rm'])
df_train_rm_2 = df_train.merge(df_rm, how='left', left_on=['cfips', 'year'], right_on=['fips','year_lag_2'], 
                              indicator=True, suffixes=['_orig', 'rm'])
df_train_rm_2

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year_orig,year_month,fips,yearrm,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,year_lag_1,year_lag_2,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,1001,2017,633,735,886,1136,1536,2018,2019,both
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,201909,1001,2017,633,735,886,1136,1536,2018,2019,both
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,201910,1001,2017,633,735,886,1136,1536,2018,2019,both
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,201911,1001,2017,633,735,886,1136,1536,2018,2019,both
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,201912,1001,2017,633,735,886,1136,1536,2018,2019,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182349,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,202206,56045,2020,696,707,931,1160,1635,2021,2022,both
182350,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,202207,56045,2020,696,707,931,1160,1635,2021,2022,both
182351,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208,56045,2020,696,707,931,1160,1635,2021,2022,both
182352,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,202209,56045,2020,696,707,931,1160,1635,2021,2022,both


In [429]:
df_train_rm = df_train_rm[['cfips', 'year', 'microbusiness_density', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

In [430]:
rm_values = ['cfips', 'year_lag_2', 'microbusiness_density', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']
for i in rm_values:
    print(i, ': ', df_train_rm_2[i].isna().sum())

cfips :  0
year_lag_2 :  0
microbusiness_density :  0
rent50_0 :  0
rent50_1 :  0
rent50_2 :  0
rent50_3 :  0
rent50_4 :  0


In [431]:
df_train_rm_2.corr()

,cfips,microbusiness_density,active,year_orig,fips,yearrm,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,year_lag_1,year_lag_2
cfips,1.000000,-0.012149,-0.121320,0.000109,1.000000,0.000109,-0.060677,-0.064838,-0.073080,-0.070825,-0.061677,0.000109,0.000109
microbusiness_density,-0.012149,1.000000,0.376921,0.011894,-0.012149,0.011894,0.466489,0.481625,0.482682,0.492490,0.492254,0.011894,0.011894
active,-0.121320,0.376921,1.000000,0.004278,-0.121320,0.004278,0.497707,0.519345,0.521511,0.516460,0.502913,0.004278,0.004278
year_orig,0.000109,0.011894,0.004278,1.000000,0.000109,1.000000,0.085992,0.079567,0.079132,0.075136,0.074570,1.000000,1.000000
fips,1.000000,-0.012149,-0.121320,0.000109,1.000000,0.000109,-0.060677,-0.064838,-0.073080,-0.070825,-0.061677,0.000109,0.000109
yearrm,0.000109,0.011894,0.004278,1.000000,0.000109,1.000000,0.085992,0.079567,0.079132,0.075136,0.074570,1.000000,1.000000
rent50_0,-0.060677,0.466489,0.497707,0.085992,-0.060677,0.085992,1.000000,0.978835,0.970643,0.960879,0.937645,0.085992,0.085992
rent50_1,-0.064838,0.481625,0.519345,0.079567,-0.064838,0.079567,0.978835,1.000000,0.990953,0.980865,0.957100,0.079567,0.079567
rent50_2,-0.073080,0.482682,0.521511,0.079132,-0.073080,0.079132,0.970643,0.990953,1.000000,0.989475,0.963995,0.079132,0.079132
rent50_3,-0.070825,0.492490,0.516460,0.075136,-0.070825,0.075136,0.960879,0.980865,0.989475,1.000000,0.974950,0.075136,0.075136


In [432]:
vars_list = ['rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']
vars_combinations = list(combinations(vars_list, 5))
vars_combinations = [list(tup) for tup in vars_combinations]
#print(vars_combinations)

for lists in vars_combinations:
    x = df_train_rm[lists]
    y = df_train_rm[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())


                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.250
Model:                               OLS   Adj. R-squared:                  0.250
Method:                    Least Squares   F-statistic:                 1.213e+04
Date:                   Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                           19:45:51   Log-Likelihood:            -5.1022e+05
No. Observations:                 182293   AIC:                         1.020e+06
Df Residuals:                     182287   BIC:                         1.021e+06
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7104 

In [433]:
df_me_yearly = df_me[(df_me['MONTH'] == 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)

df_me_monthly = df_me[(df_me['MONTH'] != 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)

C:\Users\jacks\AppData\Local\Temp\ipykernel_17660\3095542973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)
C:\Users\jacks\AppData\Local\Temp\ipykernel_17660\3095542973.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)


After looking through multiple regrison analysis we have decided to keep all of 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4' with no lags

In [434]:
df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')
df_me_monthly

C:\Users\jacks\AppData\Local\Temp\ipykernel_17660\3606587767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')


,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
2,2022,April,County,AL,Alabama,Autauga,1001,11492,5150,6342,...,2339,55,2284,10643,4303,6340,7985,1829,6156,2022-04-01
3,2022,April,County,AL,Alabama,Baldwin,1003,58902,27624,31278,...,6798,131,6667,55097,23830,31267,44841,15131,29710,2022-04-01
4,2022,April,County,AL,Alabama,Barbour,1005,6260,2440,3820,...,1329,41,1288,5932,2113,3819,5050,1396,3654,2022-04-01
5,2022,April,County,AL,Alabama,Bibb,1007,4659,1710,2949,...,1140,23,1117,4369,1420,2949,3622,891,2731,2022-04-01
6,2022,April,County,AL,Alabama,Blount,1009,13230,4979,8251,...,2431,28,2403,12439,4188,8251,10720,2715,8005,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60033,2021,September,County,PR,Puerto Rico,Unknown,72999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-09-01
60035,2021,September,County,VI,Virgin Islands,St. Croix,78010,9838,7321,2517,...,769,31,738,8913,6396,2517,5366,2867,2499,2021-09-01
60036,2021,September,County,VI,Virgin Islands,St. John,78020,921,745,176,...,47,*,*,838,662,176,470,299,171,2021-09-01
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030,9625,6669,2956,...,669,50,619,8675,5719,2956,5792,2859,2933,2021-09-01


In [435]:
df_me_monthly['date'].max()

Timestamp('2022-09-01 00:00:00')

In [436]:
df_me_monthly.groupby(['BENE_FIPS_CD'])['date'].max().min()

Timestamp('2022-09-01 00:00:00')

In [437]:
df_me_monthly.groupby(['BENE_FIPS_CD'])['date'].min().max()

Timestamp('2021-09-01 00:00:00')

In [438]:
date_df = pd.DataFrame(pd.date_range(start='2017-01-01', end='2021-08-01'), columns=['date'])

In [439]:
date_df['year'] = date_df.date.dt.year
date_df['day'] = date_df.date.dt.day
date_df = date_df[date_df.day == 1]

In [440]:
df_me_yearly

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES
40034,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,7998,2463,56,2407,9810,5085,4725,7009,2396,4613
40035,2017,Year,County,AL,Alabama,Baldwin,1003,48910,28388,20522,...,41444,7320,113,7208,45818,25302,20516,34959,15298,19661
40036,2017,Year,County,AL,Alabama,Barbour,1005,6275,4372,1903,...,4654,1569,54,1515,5878,3975,1903,4649,2828,1821
40037,2017,Year,County,AL,Alabama,Bibb,1007,4840,2480,2360,...,3369,1451,34,1416,4553,2194,2359,3602,1403,2199
40038,2017,Year,County,AL,Alabama,Blount,1009,12286,6070,6215,...,9504,2744,22,2722,11570,5355,6215,9583,3523,6060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56697,2021,Year,County,PR,Puerto Rico,Unknown,72999,*,*,*,...,*,*,*,*,*,*,*,*,*,*
56699,2021,Year,County,VI,Virgin Islands,St. Croix,78010,9788,7292,2496,...,8958,761,31,730,8865,6369,2496,5367,2889,2479
56700,2021,Year,County,VI,Virgin Islands,St. John,78020,907,735,172,...,*,46,*,*,823,651,172,467,301,167
56701,2021,Year,County,VI,Virgin Islands,St. Thomas,78030,9596,6664,2932,...,8854,666,50,616,8636,5704,2932,5793,2883,2910


In [441]:
df_me_yearly_month = df_me_yearly.merge(date_df, how="cross")
df_me_yearly_month = df_me_yearly_month.drop(['year', 'day'], axis=1)
df_me_yearly_month

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
0,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-01-01
1,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-02-01
2,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-03-01
3,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-04-01
4,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917163,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-04-01
917164,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-05-01
917165,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-06-01
917166,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-07-01


In [442]:
df_me_monthly

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
2,2022,April,County,AL,Alabama,Autauga,1001,11492,5150,6342,...,2339,55,2284,10643,4303,6340,7985,1829,6156,2022-04-01
3,2022,April,County,AL,Alabama,Baldwin,1003,58902,27624,31278,...,6798,131,6667,55097,23830,31267,44841,15131,29710,2022-04-01
4,2022,April,County,AL,Alabama,Barbour,1005,6260,2440,3820,...,1329,41,1288,5932,2113,3819,5050,1396,3654,2022-04-01
5,2022,April,County,AL,Alabama,Bibb,1007,4659,1710,2949,...,1140,23,1117,4369,1420,2949,3622,891,2731,2022-04-01
6,2022,April,County,AL,Alabama,Blount,1009,13230,4979,8251,...,2431,28,2403,12439,4188,8251,10720,2715,8005,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60033,2021,September,County,PR,Puerto Rico,Unknown,72999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-09-01
60035,2021,September,County,VI,Virgin Islands,St. Croix,78010,9838,7321,2517,...,769,31,738,8913,6396,2517,5366,2867,2499,2021-09-01
60036,2021,September,County,VI,Virgin Islands,St. John,78020,921,745,176,...,47,*,*,838,662,176,470,299,171,2021-09-01
60037,2021,September,County,VI,Virgin Islands,St. Thomas,78030,9625,6669,2956,...,669,50,619,8675,5719,2956,5792,2859,2933,2021-09-01


In [443]:
df_me_final = pd.concat([df_me_monthly, df_me_yearly_month])
df_me_final = df_me_final.replace({'*': np.nan})
print(df_me_final.columns)
df_me_final

Index(['YEAR', 'MONTH', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'AGED_ESRD_BENES',
       'AGED_NO_ESRD_BENES', 'DSBLD_TOT_BENES',
       'DSBLD_ESRD_AND_ESRD_ONLY_BENES', 'DSBLD_NO_ESRD_BENES',
       'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES', 'A_B_MA_AND_OTH_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'PRSCRPTN_DRUG_PDP_BENES',
       'PRSCRPTN_DRUG_MAPD_BENES', 'date'],
      dtype='object')


,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
2,2022,April,County,AL,Alabama,Autauga,1001,11492,5150,6342,...,2339,55,2284,10643,4303,6340,7985,1829,6156,2022-04-01
3,2022,April,County,AL,Alabama,Baldwin,1003,58902,27624,31278,...,6798,131,6667,55097,23830,31267,44841,15131,29710,2022-04-01
4,2022,April,County,AL,Alabama,Barbour,1005,6260,2440,3820,...,1329,41,1288,5932,2113,3819,5050,1396,3654,2022-04-01
5,2022,April,County,AL,Alabama,Bibb,1007,4659,1710,2949,...,1140,23,1117,4369,1420,2949,3622,891,2731,2022-04-01
6,2022,April,County,AL,Alabama,Blount,1009,13230,4979,8251,...,2431,28,2403,12439,4188,8251,10720,2715,8005,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917163,2021,Year,County,VI,Virgin Islands,Unknown,78999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-01
917164,2021,Year,County,VI,Virgin Islands,Unknown,78999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-01
917165,2021,Year,County,VI,Virgin Islands,Unknown,78999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-01
917166,2021,Year,County,VI,Virgin Islands,Unknown,78999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01


In [444]:
me_values = ['YEAR', 'MONTH', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'AGED_ESRD_BENES',
       'AGED_NO_ESRD_BENES', 'DSBLD_TOT_BENES',
       'DSBLD_ESRD_AND_ESRD_ONLY_BENES', 'DSBLD_NO_ESRD_BENES',
       'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES', 'A_B_MA_AND_OTH_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'PRSCRPTN_DRUG_PDP_BENES',
       'PRSCRPTN_DRUG_MAPD_BENES', 'date']
for i in me_values:
    print(i, ': ', df_me_final[i].isna().sum())

YEAR :  0
MONTH :  0
BENE_GEO_LVL :  0
BENE_STATE_ABRVTN :  0
BENE_STATE_DESC :  0
BENE_COUNTY_DESC :  0
BENE_FIPS_CD :  0
TOT_BENES :  15826
ORGNL_MDCR_BENES :  31651
MA_AND_OTH_BENES :  31651
AGED_TOT_BENES :  22806
AGED_ESRD_BENES :  274882
AGED_NO_ESRD_BENES :  274882
DSBLD_TOT_BENES :  22806
DSBLD_ESRD_AND_ESRD_ONLY_BENES :  313365
DSBLD_NO_ESRD_BENES :  313365
A_B_TOT_BENES :  15830
A_B_ORGNL_MDCR_BENES :  31655
A_B_MA_AND_OTH_BENES :  31655
PRSCRPTN_DRUG_TOT_BENES :  16123
PRSCRPTN_DRUG_PDP_BENES :  50457
PRSCRPTN_DRUG_MAPD_BENES :  50457
date :  0


In [445]:
df_me_final_2 = df_me_final[['YEAR', 'MONTH', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES', 'A_B_MA_AND_OTH_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'PRSCRPTN_DRUG_PDP_BENES',
       'PRSCRPTN_DRUG_MAPD_BENES', 'date']]

In [488]:
df_me_final_2['date_lag_9'] = df_me_final_2['date'] + pd.DateOffset(months=9)
df_me_final_2['date_lag_10'] = df_me_final_2['date'] + pd.DateOffset(months=10)
df_me_final_2['date_lag_11'] = df_me_final_2['date'] + pd.DateOffset(months=11)
df_me_final_2['date_lag_12'] = df_me_final_2['date'] + pd.DateOffset(months=12)
df_me_final_2['date_lag_24'] = df_me_final_2['date'] + pd.DateOffset(months=24)

In [489]:
df_train_me9 = df_train.merge(df_me_final_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['BENE_FIPS_CD','date_lag_9'], 
                              indicator=True, suffixes=['_orig', '_me'])

df_train_me10 = df_train.merge(df_me_final_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['BENE_FIPS_CD','date_lag_10'], 
                              indicator=True, suffixes=['_orig', '_me'])

df_train_me11 = df_train.merge(df_me_final_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['BENE_FIPS_CD','date_lag_11'], 
                              indicator=True, suffixes=['_orig', '_me'])

df_train_me12 = df_train.merge(df_me_final_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['BENE_FIPS_CD','date_lag_12'], 
                              indicator=True, suffixes=['_orig', '_me'])
df_train_me24 = df_train.merge(df_me_final_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['BENE_FIPS_CD','date_lag_24'], 
                              indicator=True, suffixes=['_orig', '_me'])

df_train_me12

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,year_month,YEAR,...,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date,date_lag_9,date_lag_10,date_lag_11,date_lag_12,date_lag_24,_merge
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,2017,...,7009,2396,4613,2018-08-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2020-08-01,both
1,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,2018,...,7088,2260,4828,2018-08-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2020-08-01,both
2,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,2019,...,7414,2242,5172,2018-08-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2020-08-01,both
3,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,2020,...,7612,2101,5512,2018-08-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2020-08-01,both
4,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,201908,2021,...,7838,1931,5907,2018-08-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2020-08-01,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586166,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208,2019,...,1067,984,84,2021-08-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2023-08-01,both
586167,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208,2020,...,1100,1006,94,2021-08-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2023-08-01,both
586168,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,202208,2021,...,1133,1016,117,2021-08-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2023-08-01,both
586169,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,202209,2021,...,1135,1017,118,2021-09-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2023-09-01,both


In [448]:
df_train_me12.columns

Index(['row_id', 'cfips', 'county', 'state', 'first_day_of_month',
       'microbusiness_density', 'active', 'year', 'year_month', 'YEAR',
       'MONTH', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES', 'A_B_MA_AND_OTH_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'PRSCRPTN_DRUG_PDP_BENES',
       'PRSCRPTN_DRUG_MAPD_BENES', 'date', 'date_lag_9', 'date_lag_10',
       'date_lag_11', 'date_lag_12', '_merge'],
      dtype='object')

In [451]:
me_values = ['YEAR', 'MONTH', 'microbusiness_density', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES', 'A_B_MA_AND_OTH_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'PRSCRPTN_DRUG_PDP_BENES',
       'PRSCRPTN_DRUG_MAPD_BENES', 'date_lag_12']
for i in me_values:
    print(i, ': ', df_train_me12[i].isna().sum())

YEAR :  0
MONTH :  0
microbusiness_density :  0
BENE_GEO_LVL :  0
BENE_STATE_ABRVTN :  0
BENE_STATE_DESC :  0
BENE_COUNTY_DESC :  0
BENE_FIPS_CD :  0
TOT_BENES :  113
ORGNL_MDCR_BENES :  10201
MA_AND_OTH_BENES :  10201
AGED_TOT_BENES :  4346
DSBLD_TOT_BENES :  4346
A_B_TOT_BENES :  113
A_B_ORGNL_MDCR_BENES :  10201
A_B_MA_AND_OTH_BENES :  10201
PRSCRPTN_DRUG_TOT_BENES :  300
PRSCRPTN_DRUG_PDP_BENES :  21987
PRSCRPTN_DRUG_MAPD_BENES :  21987
date_lag_12 :  0


In [490]:
df_train_me12_r = df_train_me12[['microbusiness_density', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']].dropna()
df_train_me24_r = df_train_me24[['microbusiness_density', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']].dropna()
df_train_me11_r = df_train_me11[['microbusiness_density', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']].dropna()
df_train_me10_r = df_train_me10[['microbusiness_density', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']].dropna()
df_train_me9_r = df_train_me9[['microbusiness_density', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']].dropna()

In [478]:
df_train_me12_r

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
0,3.007682,10510,8046,2463,9810,7009
1,3.007682,10645,8241,2403,9892,7088
2,3.007682,11059,8596,2463,10249,7414
3,3.007682,11251,8840,2411,10425,7612
4,3.007682,11396,9031,2365,10555,7838
...,...,...,...,...,...,...
586166,1.785395,1629,1454,175,1510,1067
586167,1.785395,1664,1496,168,1538,1100
586168,1.785395,1713,1555,158,1588,1133
586169,1.785395,1716,1555,161,1591,1135


In [491]:
me_values = ['TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
for i in me_values:
    df_train_me12_r[i] = df_train_me12_r[i].astype(int)
for i in me_values:
    df_train_me11_r[i] = df_train_me11_r[i].astype(int)
for i in me_values:
    df_train_me10_r[i] = df_train_me10_r[i].astype(int)
for i in me_values:
    df_train_me9_r[i] = df_train_me9_r[i].astype(int)
for i in me_values:
    df_train_me24_r[i] = df_train_me24_r[i].astype(int)

In [480]:
df_train_me12_r.corr()

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
microbusiness_density,1.000000,0.281662,0.284285,0.249021,0.282414,0.270637
TOT_BENES,0.281662,1.000000,0.999274,0.958901,0.999682,0.997346
AGED_TOT_BENES,0.284285,0.999274,1.000000,0.947397,0.998751,0.997178
DSBLD_TOT_BENES,0.249021,0.958901,0.947397,1.000000,0.960125,0.952218
A_B_TOT_BENES,0.282414,0.999682,0.998751,0.960125,1.000000,0.997460
PRSCRPTN_DRUG_TOT_BENES,0.270637,0.997346,0.997178,0.952218,0.997460,1.000000


In [483]:
df_train_me11_r.corr()

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
microbusiness_density,1.000000,0.281670,0.284296,0.248989,0.282419,0.270654
TOT_BENES,0.281670,1.000000,0.999275,0.958871,0.999682,0.997347
AGED_TOT_BENES,0.284296,0.999275,1.000000,0.947367,0.998752,0.997180
DSBLD_TOT_BENES,0.248989,0.958871,0.947367,1.000000,0.960093,0.952187
A_B_TOT_BENES,0.282419,0.999682,0.998752,0.960093,1.000000,0.997463
PRSCRPTN_DRUG_TOT_BENES,0.270654,0.997347,0.997180,0.952187,0.997463,1.000000


In [484]:
df_train_me10_r.corr()

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
microbusiness_density,1.000000,0.281642,0.284271,0.248921,0.282387,0.270636
TOT_BENES,0.281642,1.000000,0.999275,0.958842,0.999682,0.997349
AGED_TOT_BENES,0.284271,0.999275,1.000000,0.947339,0.998753,0.997182
DSBLD_TOT_BENES,0.248921,0.958842,0.947339,1.000000,0.960063,0.952157
A_B_TOT_BENES,0.282387,0.999682,0.998753,0.960063,1.000000,0.997465
PRSCRPTN_DRUG_TOT_BENES,0.270636,0.997349,0.997182,0.952157,0.997465,1.000000


In [485]:
df_train_me9_r.corr()

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
microbusiness_density,1.000000,0.281666,0.284296,0.248890,0.282408,0.270672
TOT_BENES,0.281666,1.000000,0.999275,0.958762,0.999682,0.997351
AGED_TOT_BENES,0.284296,0.999275,1.000000,0.947249,0.998754,0.997185
DSBLD_TOT_BENES,0.248890,0.958762,0.947249,1.000000,0.959978,0.952061
A_B_TOT_BENES,0.282408,0.999682,0.998754,0.959978,1.000000,0.997468
PRSCRPTN_DRUG_TOT_BENES,0.270672,0.997351,0.997185,0.952061,0.997468,1.000000


In [492]:
df_train_me24_r.corr()

,microbusiness_density,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
microbusiness_density,1.000000,0.280216,0.282825,0.247789,0.280971,0.269235
TOT_BENES,0.280216,1.000000,0.999274,0.958957,0.999682,0.997343
AGED_TOT_BENES,0.282825,0.999274,1.000000,0.947453,0.998749,0.997174
DSBLD_TOT_BENES,0.247789,0.958957,0.947453,1.000000,0.960185,0.952276
A_B_TOT_BENES,0.280971,0.999682,0.998749,0.960185,1.000000,0.997456
PRSCRPTN_DRUG_TOT_BENES,0.269235,0.997343,0.997174,0.952276,0.997456,1.000000


In [496]:
vars_list = ['TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
vars_combinations = list(combinations(vars_list, 5))
vars_combinations = [list(tup) for tup in vars_combinations]
#print(vars_combinations)

for lists in vars_combinations:
    x = df_train_me12_r[lists]
    y = df_train_me12_r[['microbusiness_density']]

    x = sm.add_constant(x) # adding a constant

    model = sm.OLS(y, x).fit()

    print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     microbusiness_density   R-squared:                       0.120
Model:                               OLS   Adj. R-squared:                  0.120
Method:                    Least Squares   F-statistic:                 1.593e+04
Date:                   Sun, 12 Feb 2023   Prob (F-statistic):               0.00
Time:                           20:47:12   Log-Likelihood:            -1.7198e+06
No. Observations:                 581712   AIC:                         3.440e+06
Df Residuals:                     581706   BIC:                         3.440e+06
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------

We keep all of all 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES' with a 12 month lag